In [76]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle
import re

In [77]:
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils
sys.setrecursionlimit(40000)

In [78]:
from keras_frcnn.simple_parser import get_data
from keras_frcnn import resnet as nn

In [79]:
# Create a config object and set the values
C = config.Config()
firstTraining = True

C.use_horizontal_flips = bool(False)
C.use_vertical_flips = bool(False)
C.rot_90 = bool(False)
C.model_path = './coco_model_frcnn_airplane_vehicle_v1.hdf5'
C.verbose = True
model_path_regex = re.match("^(.+)(\.hdf5)$", C.model_path)
if model_path_regex.group(2) != '.hdf5':
	print('Output weights must have .hdf5 filetype')
	exit(1)
C.num_rois = int(32)
C.network = 'resnet50'
if firstTraining is True:
    C.base_net_weights = nn.get_weight_path()
    firstTraining = False
else:
    C.base_net_weights = options.input_weight_path
config_output_filename = "coco_config_airplane.pickle"
#train_path = './annotate.txt'
#train_path = 'COCO_extracted_dataset/airplane_for_train/coco_airplane_annotate.txt'
train_path = 'coco_training_Airplane_annotate.txt'
val_path = 'coco_validation_Airplane_annotate.txt'

In [80]:
train_imgs, classes_count, class_mapping = get_data(train_path)
val_imgs, _, _ = get_data(val_path)

if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping
inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print(f'Num classes (including bg) = {len(classes_count)}')

Parsing annotation files
Found class name with special name bg. Will be treated as a background region (this is usually for hard negative mining).
Parsing annotation files
Training images per class:
{'airplane': 950, 'bg': 50}
Num classes (including bg) = 2


In [81]:
train_imgs

[{'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000393221.jpg',
  'width': 427,
  'height': 640,
  'bboxes': [{'class': 'airplane',
    'x1': 205.59,
    'x2': 208.51,
    'y1': 52.94,
    'y2': 57.93}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000376838.jpg',
  'width': 500,
  'height': 377,
  'bboxes': [{'class': 'airplane',
    'x1': 311.71,
    'x2': 487.78,
    'y1': 156.23,
    'y2': 246.78}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000352264.jpg',
  'width': 640,
  'height': 480,
  'bboxes': [{'class': 'airplane',
    'x1': 97.52,
    'x2': 556.9300000000001,
    'y1': 124.33,
    'y2': 297.7}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000024585.jpg',
  'width': 640,
  'height': 432,
  'bboxes': [{'class': 'airplane',
    'x1': 178.79,
    'x2': 632.05,
    'y1': 126.74,
    'y2': 34

In [82]:
val_imgs

[{'filepath': 'COCO_extracted_dataset/airplane_for_validation/COCO_val2014_000000081922.jpg',
  'width': 640,
  'height': 426,
  'bboxes': [{'class': 'airplane',
    'x1': 218.07,
    'x2': 377.0,
    'y1': 183.32,
    'y2': 230.22}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_validation/COCO_val2014_000000262148.jpg',
  'width': 640,
  'height': 512,
  'bboxes': [{'class': 'airplane',
    'x1': 185.77,
    'x2': 197.57,
    'y1': 189.62,
    'y2': 198.28}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_validation/COCO_val2014_000000208901.jpg',
  'width': 640,
  'height': 427,
  'bboxes': [{'class': 'airplane',
    'x1': 232.67,
    'x2': 425.99,
    'y1': 148.37,
    'y2': 284.5900000000001}],
  'imageset': 'test'},
 {'filepath': 'COCO_extracted_dataset/airplane_for_validation/COCO_val2014_000000184330.jpg',
  'width': 640,
  'height': 425,
  'bboxes': [{'class': 'airplane',
    'x1': 241.46,
    'x2': 454.16,
    'y1': 229.5

In [83]:
class_mapping

{'airplane': 0, 'bg': 1}

In [84]:
with open(config_output_filename, 'wb') as config_f:
	pickle.dump(C,config_f)
	print(f'Config has been written to {config_output_filename}, and can be loaded when testing to ensure correct results')

#random.shuffle(train_imgs)

num_imgs = len(train_imgs)
print(f'Num train samples {len(train_imgs)}')
print(f'Num val samples {len(val_imgs)}')

Config has been written to coco_config_airplane.pickle, and can be loaded when testing to ensure correct results
Num train samples 1000
Num val samples 200


In [85]:
data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.common.image_dim_ordering(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.common.image_dim_ordering(), mode='val')


In [86]:
data_gen_train

<generator object get_anchor_gt at 0x0000014D9A377678>

In [87]:
data_gen_val

<generator object get_anchor_gt at 0x0000014D9A377620>

In [88]:
if K.common.image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
else:
	input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

In [89]:
img_input

<tf.Tensor 'input_5:0' shape=(None, None, None, 3) dtype=float32>

In [90]:
roi_input

<tf.Tensor 'input_6:0' shape=(None, None, 4) dtype=float32>

In [91]:
# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)
shared_layers

<tf.Tensor 'activation_138/Relu:0' shape=(None, None, None, 1024) dtype=float32>

In [92]:
C.anchor_box_scales

[128, 256, 512]

In [93]:
C.anchor_box_ratios

[[1, 1],
 [0.7071067811865475, 1.414213562373095],
 [1.414213562373095, 0.7071067811865475]]

In [94]:
# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
num_anchors

9

In [95]:
rpn = nn.rpn(shared_layers, num_anchors)
rpn

[<tf.Tensor 'rpn_out_class_2/Sigmoid:0' shape=(None, None, None, 9) dtype=float32>,
 <tf.Tensor 'rpn_out_regress_2/BiasAdd:0' shape=(None, None, None, 36) dtype=float32>,
 <tf.Tensor 'activation_138/Relu:0' shape=(None, None, None, 1024) dtype=float32>]

In [96]:
classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)
classifier

[<tf.Tensor 'dense_class_2_2/Reshape_1:0' shape=(1, 32, 2) dtype=float32>,
 <tf.Tensor 'dense_regress_2_2/Reshape_1:0' shape=(1, 32, 4) dtype=float32>]

In [97]:
model_rpn = Model(img_input, rpn[:2])
model_rpn.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [98]:
model_classifier = Model([img_input, roi_input], classifier)
model_classifier.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [99]:
# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)
model_all.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [100]:
try:
	print('loading weights from {C.base_net_weights}')
	model_rpn.load_weights(C.base_net_weights, by_name=True)
	model_classifier.load_weights(C.base_net_weights, by_name=True)    
	#model_rpn.load_weights('./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)
	#model_classifier.load_weights('./resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')

loading weights from {C.base_net_weights}


In [101]:
optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={f'dense_class_{len(classes_count)}': 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

In [102]:
model_rpn.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [103]:
model_classifier.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [104]:
model_all.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, None, None, 3 0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
bn_conv1 (FixedBatchNormalizati (None, None, None, 6 256         conv1[0][0]                      
____________________________________________________________________________________________

In [105]:
#number of images in the dataset
epoch_length = 1000 
num_epochs = int(5)
iter_num = 0

In [106]:
losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
start_time = time.time()

best_loss = np.Inf

In [107]:
class_mapping_inv = {v: k for k, v in class_mapping.items()}
print('Starting training')

vis = True
index = 0

Starting training


In [108]:
for epoch_num in range(num_epochs):

	progbar = generic_utils.Progbar(epoch_length)
	print(f'Epoch {epoch_num + 1}/{num_epochs}')

	while True:
		try:

			if len(rpn_accuracy_rpn_monitor) == epoch_length and C.verbose:
				mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
				rpn_accuracy_rpn_monitor = []
				print(f'Average number of overlapping bounding boxes from RPN = {mean_overlapping_bboxes} for {epoch_length} previous iterations')
				if mean_overlapping_bboxes == 0:
					print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')

			X, Y, img_data = next(data_gen_train)
			print(f'X is {X.shape}')
			print(f'Y is {len(Y)}')
			print(f'img_data is {img_data}')
			print(f'index is {index}')
			index = index + 1

			loss_rpn = model_rpn.train_on_batch(X, Y)

			P_rpn = model_rpn.predict_on_batch(X)

			R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.common.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
			# note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
			X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

			if X2 is None:
				rpn_accuracy_rpn_monitor.append(0)
				rpn_accuracy_for_epoch.append(0)
				continue

			neg_samples = np.where(Y1[0, :, -1] == 1)
			pos_samples = np.where(Y1[0, :, -1] == 0)

			if len(neg_samples) > 0:
				neg_samples = neg_samples[0]
			else:
				neg_samples = []

			if len(pos_samples) > 0:
				pos_samples = pos_samples[0]
			else:
				pos_samples = []
			
			rpn_accuracy_rpn_monitor.append(len(pos_samples))
			rpn_accuracy_for_epoch.append((len(pos_samples)))

			if C.num_rois > 1:
				if len(pos_samples) < C.num_rois//2:
					selected_pos_samples = pos_samples.tolist()
				else:
					selected_pos_samples = np.random.choice(pos_samples, C.num_rois//2, replace=False).tolist()
				try:
					selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
				except:
					selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

				sel_samples = selected_pos_samples + selected_neg_samples
			else:
				# in the extreme case where num_rois = 1, we pick a random pos or neg sample
				selected_pos_samples = pos_samples.tolist()
				selected_neg_samples = neg_samples.tolist()
				if np.random.randint(0, 2):
					sel_samples = random.choice(neg_samples)
				else:
					sel_samples = random.choice(pos_samples)

			loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

			losses[iter_num, 0] = loss_rpn[1]
			losses[iter_num, 1] = loss_rpn[2]

			losses[iter_num, 2] = loss_class[1]
			losses[iter_num, 3] = loss_class[2]
			losses[iter_num, 4] = loss_class[3]

			progbar.update(iter_num+1, [('rpn_cls', losses[iter_num, 0]), ('rpn_regr', losses[iter_num, 1]),
									  ('detector_cls', losses[iter_num, 2]), ('detector_regr', losses[iter_num, 3])])

			iter_num += 1
			
			if iter_num == epoch_length:
				loss_rpn_cls = np.mean(losses[:, 0])
				loss_rpn_regr = np.mean(losses[:, 1])
				loss_class_cls = np.mean(losses[:, 2])
				loss_class_regr = np.mean(losses[:, 3])
				class_acc = np.mean(losses[:, 4])

				mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
				rpn_accuracy_for_epoch = []

				if C.verbose:
					print(f'Mean number of bounding boxes from RPN overlapping ground truth boxes: {mean_overlapping_bboxes}')
					print(f'Classifier accuracy for bounding boxes from RPN: {class_acc}')
					print(f'Loss RPN classifier: {loss_rpn_cls}')
					print(f'Loss RPN regression: {loss_rpn_regr}')
					print(f'Loss Detector classifier: {loss_class_cls}')
					print(f'Loss Detector regression: {loss_class_regr}')
					print(f'Elapsed time: {time.time() - start_time}')

				curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
				iter_num = 0
				start_time = time.time()

				if curr_loss < best_loss:
					if C.verbose:
						print(f'Total loss decreased from {best_loss} to {curr_loss}, saving weights')
					best_loss = curr_loss
				model_all.save_weights(model_path_regex.group(1) + "_" + '{:04d}'.format(epoch_num) + model_path_regex.group(2))

				break

		except Exception as e:
			print(f'Exception: {e}')
			continue

print('Training complete, exiting.')

Epoch 1/5
X is (1, 416, 626, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000340258.jpg', 'width': 640, 'height': 425, 'bboxes': [{'class': 'airplane', 'x1': 2.85, 'x2': 631.32, 'y1': 86.61, 'y2': 328.11}], 'imageset': 'test'}
index is 0
   1/1000 [..............................] - ETA: 8:48:24 - rpn_cls: 5.1112 - rpn_regr: 0.3901 - detector_cls: 0.6931 - detector_regr: 0.5935X is (1, 416, 605, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000395850.jpg', 'width': 640, 'height': 440, 'bboxes': [{'class': 'airplane', 'x1': 81.08, 'x2': 533.9300000000001, 'y1': 36.58, 'y2': 420.22}], 'imageset': 'test'}
index is 1
   2/1000 [..............................] - ETA: 5:04:30 - rpn_cls: 4.9132 - rpn_regr: 0.4452 - detector_cls: 0.6901 - detector_regr: 0.4746X is (1, 416, 485, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000188749.jpg', 'width'

  19/1000 [..............................] - ETA: 2:09:37 - rpn_cls: 5.4999 - rpn_regr: 0.3303 - detector_cls: 0.6144 - detector_regr: 0.5155X is (1, 416, 739, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000275316.jpg', 'width': 640, 'height': 360, 'bboxes': [{'class': 'airplane', 'x1': 277.0, 'x2': 400.21, 'y1': 39.0, 'y2': 112.81}], 'imageset': 'test'}
index is 20
  20/1000 [..............................] - ETA: 2:09:06 - rpn_cls: 5.5523 - rpn_regr: 0.3194 - detector_cls: 0.5964 - detector_regr: 0.5144X is (1, 416, 623, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000540695.jpg', 'width': 640, 'height': 427, 'bboxes': [{'class': 'airplane', 'x1': 76.59, 'x2': 537.1, 'y1': 123.31, 'y2': 324.37}], 'imageset': 'test'}
index is 21
  21/1000 [..............................] - ETA: 2:07:47 - rpn_cls: 5.4284 - rpn_regr: 0.3081 - detector_cls: 0.5979 - detector_regr: 0.5057X is (1, 416, 62

  37/1000 [>.............................] - ETA: 2:00:47 - rpn_cls: 4.5634 - rpn_regr: 0.3165 - detector_cls: 0.5620 - detector_regr: 0.4626X is (1, 416, 626, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000361519.jpg', 'width': 500, 'height': 332, 'bboxes': [{'class': 'airplane', 'x1': 0.0, 'x2': 135.39, 'y1': 0.0, 'y2': 27.9}], 'imageset': 'test'}
index is 40
  38/1000 [>.............................] - ETA: 2:00:32 - rpn_cls: 4.5268 - rpn_regr: 0.3242 - detector_cls: 0.5546 - detector_regr: 0.4504X is (1, 416, 623, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000263651.jpg', 'width': 640, 'height': 427, 'bboxes': [{'class': 'airplane', 'x1': 223.0, 'x2': 474.86, 'y1': 31.77, 'y2': 96.51}], 'imageset': 'test'}
index is 41
  39/1000 [>.............................] - ETA: 2:00:02 - rpn_cls: 4.5612 - rpn_regr: 0.3220 - detector_cls: 0.5452 - detector_regr: 0.4554X is (1, 416, 695, 3)


  55/1000 [>.............................] - ETA: 1:55:50 - rpn_cls: 4.6711 - rpn_regr: 0.3087 - detector_cls: 0.5329 - detector_regr: 0.4436X is (1, 416, 623, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000404740.jpg', 'width': 640, 'height': 427, 'bboxes': [{'class': 'airplane', 'x1': 81.66, 'x2': 555.82, 'y1': 146.81, 'y2': 303.08000000000004}], 'imageset': 'test'}
index is 60
  56/1000 [>.............................] - ETA: 1:55:45 - rpn_cls: 4.5884 - rpn_regr: 0.3046 - detector_cls: 0.5329 - detector_regr: 0.4405X is (1, 416, 554, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000231902.jpg', 'width': 640, 'height': 480, 'bboxes': [{'class': 'airplane', 'x1': 146.09, 'x2': 354.58000000000004, 'y1': 171.13, 'y2': 254.31}], 'imageset': 'test'}
index is 61
  57/1000 [>.............................] - ETA: 1:55:25 - rpn_cls: 4.5303 - rpn_regr: 0.3002 - detector_cls: 0.5299 - detector_

  72/1000 [=>............................] - ETA: 1:54:15 - rpn_cls: 4.2098 - rpn_regr: 0.2887 - detector_cls: 0.5212 - detector_regr: 0.4402X is (1, 416, 605, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000445845.jpg', 'width': 640, 'height': 440, 'bboxes': [{'class': 'airplane', 'x1': 34.32, 'x2': 584.3900000000001, 'y1': 129.65, 'y2': 289.0}], 'imageset': 'test'}
index is 80
  73/1000 [=>............................] - ETA: 1:54:04 - rpn_cls: 4.1892 - rpn_regr: 0.2874 - detector_cls: 0.5208 - detector_regr: 0.4399X is (1, 416, 624, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000283426.jpg', 'width': 640, 'height': 426, 'bboxes': [{'class': 'airplane', 'x1': 63.18, 'x2': 528.43, 'y1': 116.79, 'y2': 267.09000000000003}], 'imageset': 'test'}
index is 81
  74/1000 [=>............................] - ETA: 1:53:48 - rpn_cls: 4.1353 - rpn_regr: 0.2847 - detector_cls: 0.5209 - detector_reg

...

 924/1000 [==========================>...] - ETA: 8:30 - rpn_cls: 0.5443 - rpn_regr: 0.1719 - detector_cls: 0.2098 - detector_regr: 0.1370X is (1, 416, 623, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000452799.jpg', 'width': 640, 'height': 427, 'bboxes': [{'class': 'airplane', 'x1': 402.54, 'x2': 471.0, 'y1': 212.72, 'y2': 237.16}], 'imageset': 'test'}
index is 5337
 925/1000 [==========================>...] - ETA: 8:23 - rpn_cls: 0.5438 - rpn_regr: 0.1719 - detector_cls: 0.2099 - detector_regr: 0.1375X is (1, 416, 622, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000240602.jpg', 'width': 640, 'height': 428, 'bboxes': [{'class': 'airplane', 'x1': 27.17, 'x2': 430.91, 'y1': 92.52, 'y2': 245.87}], 'imageset': 'test'}
index is 5338
 926/1000 [==========================>...] - ETA: 8:16 - rpn_cls: 0.5432 - rpn_regr: 0.1718 - detector_cls: 0.2098 - detector_regr: 0.1374X is (1, 416, 623, 

 942/1000 [===========================>..] - ETA: 6:29 - rpn_cls: 0.5346 - rpn_regr: 0.1717 - detector_cls: 0.2096 - detector_regr: 0.1370X is (1, 416, 430, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000484256.jpg', 'width': 500, 'height': 483, 'bboxes': [{'class': 'airplane', 'x1': 0.0, 'x2': 75.75, 'y1': 249.99, 'y2': 292.55}], 'imageset': 'test'}
index is 5357
 943/1000 [===========================>..] - ETA: 6:22 - rpn_cls: 0.5402 - rpn_regr: 0.1717 - detector_cls: 0.2095 - detector_regr: 0.1373X is (1, 416, 624, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000379749.jpg', 'width': 640, 'height': 426, 'bboxes': [{'class': 'airplane', 'x1': 57.18, 'x2': 562.28, 'y1': 88.63, 'y2': 259.22}], 'imageset': 'test'}
index is 5358
 944/1000 [===========================>..] - ETA: 6:16 - rpn_cls: 0.5396 - rpn_regr: 0.1715 - detector_cls: 0.2094 - detector_regr: 0.1373X is (1, 416, 554, 3)


 961/1000 [===========================>..] - ETA: 4:22 - rpn_cls: 0.5322 - rpn_regr: 0.1712 - detector_cls: 0.2091 - detector_regr: 0.1367X is (1, 416, 626, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000361519.jpg', 'width': 500, 'height': 332, 'bboxes': [{'class': 'airplane', 'x1': 0.0, 'x2': 135.39, 'y1': 0.0, 'y2': 27.9}], 'imageset': 'test'}
index is 5377
 962/1000 [===========================>..] - ETA: 4:15 - rpn_cls: 0.5366 - rpn_regr: 0.1715 - detector_cls: 0.2090 - detector_regr: 0.1368X is (1, 416, 623, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000042977.jpg', 'width': 640, 'height': 427, 'bboxes': [{'class': 'airplane', 'x1': 44.14, 'x2': 623.71, 'y1': 128.58, 'y2': 305.14}], 'imageset': 'test'}
index is 5378
 963/1000 [===========================>..] - ETA: 4:08 - rpn_cls: 0.5360 - rpn_regr: 0.1715 - detector_cls: 0.2091 - detector_regr: 0.1367X is (1, 416, 739, 3)
Y i

 981/1000 [============================>.] - ETA: 2:07 - rpn_cls: 0.5317 - rpn_regr: 0.1707 - detector_cls: 0.2088 - detector_regr: 0.1372X is (1, 416, 554, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000356869.jpg', 'width': 640, 'height': 480, 'bboxes': [{'class': 'airplane', 'x1': 26.91, 'x2': 637.13, 'y1': 4.3, 'y2': 396.05}], 'imageset': 'test'}
index is 5397
 982/1000 [============================>.] - ETA: 2:01 - rpn_cls: 0.5368 - rpn_regr: 0.1710 - detector_cls: 0.2089 - detector_regr: 0.1374X is (1, 416, 554, 3)
Y is 2
img_data is {'filepath': 'COCO_extracted_dataset/airplane_for_train/COCO_train2014_000000385589.jpg', 'width': 640, 'height': 480, 'bboxes': [{'class': 'airplane', 'x1': 160.56, 'x2': 248.12, 'y1': 155.68, 'y2': 178.69}], 'imageset': 'test'}
index is 5398
 983/1000 [============================>.] - ETA: 1:54 - rpn_cls: 0.5368 - rpn_regr: 0.1710 - detector_cls: 0.2088 - detector_regr: 0.1373X is (1, 416, 517, 3

1000/1000 [==============================] - 6730s 7s/step - rpn_cls: 0.5443 - rpn_regr: 0.1711 - detector_cls: 0.2086 - detector_regr: 0.1371
Mean number of bounding boxes from RPN overlapping ground truth boxes: 9.282894736842104
Classifier accuracy for bounding boxes from RPN: 0.90978125
Loss RPN classifier: 0.5442925147390542
Loss RPN regression: 0.17105243044893723
Loss Detector classifier: 0.20862338070677652
Loss Detector regression: 0.13709377339016646
Elapsed time: 6730.634847402573
Total loss decreased from 1.3239988268907859 to 1.0610620992849344, saving weights
Training complete, exiting.
